In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
from tqdm import tqdm
from torch.utils.data import random_split
import torch.nn.functional as F
import pandas as pd



In [2]:
torch.cuda.is_available()

False

In [3]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [4]:
def getData(path):
    train_file = np.load(path+"/train.npz")
    train_data = train_file['data']
    test_file = np.load(path+"/test_input.npz")
    test_data = test_file['data']
    print(f"Training Data's shape is {train_data.shape} and Test Data's is {test_data.shape}")
    return train_data, test_data

In [5]:
trainData, testData = getData("./data")
trainData.shape, testData.shape

Training Data's shape is (10000, 50, 110, 6) and Test Data's is (2100, 50, 50, 6)


((10000, 50, 110, 6), (2100, 50, 50, 6))

In [6]:
import torch
from torch.utils.data import Dataset

class WindowedNormalizedDataset(Dataset):
    def __init__(self, data, window_size, forecast_horizon, mean=None, std=None):
        self.data = data  # shape: (samples, features, timesteps, channels)
        self.window_size = window_size
        self.forecast_horizon = forecast_horizon
        self.mean = mean
        self.std = std

        # Precompute indices of valid (sample, t) combinations
        self.indices = []
        for sample in range(data.shape[0]):
            for t in range(data.shape[2] - window_size - forecast_horizon + 1):
                self.indices.append((sample, t))

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        sample_idx, t = self.indices[idx]

        x = self.data[sample_idx, :, t:t+self.window_size, :]
        y = self.data[sample_idx, 0, t+self.window_size:t+self.window_size+self.forecast_horizon, :2]
        # print(x.shape, y.shape)
        if self.mean is not None and self.std is not None:
            x = (x - self.mean) / self.std

        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)


dataset = WindowedNormalizedDataset(data = trainData, window_size=50, forecast_horizon=60)
# dataset.__getitem__(0)

In [7]:
class EncoderDecoderModel(nn.Module):
    def __init__(self, infeatures, outfeatures=2):
        super().__init__()
        # Encoder
        self.layer1 = nn.Linear(infeatures, 32)
        self.layer2 = nn.Linear(32, 64)
        self.layer3 = nn.Linear(64, 128)
        self.encoderlstm = nn.LSTM(128, 256, num_layers=2, batch_first=True, dropout=0.3)
        
        # Changed pooling target from 20 to 60
        self.pool = nn.AdaptiveAvgPool1d(60)
        self.dropout = nn.Dropout(0.2)
        
        # Decoder
        self.decoderlstm = nn.LSTM(256, 128, num_layers=2, batch_first=True, dropout=0.3)
        self.layer10 = nn.Linear(128, 64)
        self.layer11 = nn.Linear(64, 32)
        self.layer12 = nn.Linear(32, outfeatures)
        
        # Skip connections
        self.skip1 = nn.Linear(32, 32)
        self.skip2 = nn.Linear(64, 64)
        self.skip3 = nn.Linear(128, 128)
        self.skip4 = nn.Linear(256, 256)

    def forward(self, x):
        batch_size, channels, height, width = x.shape
        
        # Encoder
        out1 = nn.ReLU()(self.layer1(x))
        out2 = nn.ReLU()(self.layer2(out1))
        out3 = nn.ReLU()(self.layer3(out2))
        
        # LSTM processing
        tempout3 = out3.view(batch_size, -1, out3.size(-1))
        out4, _ = self.encoderlstm(tempout3)
        
        # Changed pooling to 60
        tempout4 = self.pool(out4.permute(0, 2, 1))
        tempout4 = tempout4.permute(0, 2, 1)
        lstmskip = tempout4 + self.skip4(tempout4)
        
        # Decoder LSTM
        out5, _ = self.decoderlstm(lstmskip)
        
        out3_reduced = F.adaptive_avg_pool2d(out3.permute(0, 3, 1, 2), (60, 1)).squeeze(-1).permute(0, 2, 1)
        mlpskip1 = out3_reduced + self.skip3(out5)
        out6 = nn.ReLU()(self.layer10(mlpskip1))
        
        out2_reduced = F.adaptive_avg_pool2d(out2.permute(0, 3, 1, 2), (60, 1)).squeeze(-1).permute(0, 2, 1)
        mlpskip2 = out2_reduced + self.skip2(out6)
        out7 = nn.ReLU()(self.layer11(mlpskip2))
        
        out1_reduced = F.adaptive_avg_pool2d(out1.permute(0, 3, 1, 2), (60, 1)).squeeze(-1).permute(0, 2, 1)
        mlpskip3 = out1_reduced + self.skip1(out7)
        out8 = self.layer12(mlpskip3)
        
        return out8

# Verify the output shape
def xavier_init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_normal_(m.weight)
        if m.bias is not None:
            torch.nn.init.constant_(m.bias, 0)
        
model = EncoderDecoderModel(6, 2)
model.apply(xavier_init_weights)

# test = torch.randn(128, 50, 50, 6)
# out = model(test)
# print(out.shape)  # Should output torch.Size([128, 60, 2])
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 1359906


In [9]:
# trainDataset = LargeDataset(a, b, train_mean, train_std) # testing for small dataset a, b
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

# model.load_state_dict(torch.load("./models/modelE/medium_model_2365.921588.pth"))  

trainDataLoader = DataLoader(dataset, batch_size=128, shuffle=True, num_workers=0)
# testDataLoader = DataLoader(test_dataset, batch_size=128)
model.to(device)
print(len(trainDataLoader))
# Training setup
epochs = 300
lossFn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0008)

for each_epoch in range(epochs):
    model.train()
    runningLoss = 0.0
    loop = tqdm(trainDataLoader, desc=f"Epoch [{each_epoch+1}/{epochs}]")

    for batchX, batchY in loop:
        batchX, batchY = batchX.to(device, non_blocking=True), batchY.to(device, non_blocking=True)
        output = model(batchX)
        # print(output.shape, batchY.shape)
        loss = lossFn(output, batchY)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        runningLoss += loss.item()
    # break
    avgLoss = runningLoss / len(trainDataLoader)

    # model.eval()
    # with torch.inference_mode():
    #     testloss = 0.0
    #     for testX, testY in testDataLoader:
    #         testX, testY = testX.to(device, non_blocking=True), testY.to(device, non_blocking=True)
    #         pred = model(testX)
    #         tloss = lossFn(pred, testY)
    #         testloss += tloss.item()

    #     avgtestloss = testloss/len(testDataLoader)
        # if each_epoch % 5 == 0 or each_epoch+1 >= epochs:
    print(f"Epoch {each_epoch + 1}, Training Loss: {avgLoss:.4f}")
    torch.save(model.state_dict(), f'./models/modelE/medium_model_{avgLoss:.6f}.pth')
    torch.cuda.empty_cache()


79


Epoch [1/300]:   0%|          | 0/79 [00:03<?, ?it/s]


KeyboardInterrupt: 

In [8]:
def createSubmission(modelPath, submissionFileName):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.load_state_dict(torch.load(modelPath, map_location=device))  
    test_data = torch.tensor(testData, dtype=torch.float32)
    rows = []
    for i in range(test_data.shape[0]):
        pred = model(test_data[i].unsqueeze(0))
        # print(pred.shape)
        items = pred.squeeze(0).tolist()
        for idx, (x, y) in enumerate(items):
            rows.append({'x': x, 'y': y})

    df = pd.DataFrame(rows)
    df.index.name = 'index'
    df.to_csv('./submission/'+submissionFileName)


createSubmission("./models/medium_model_496.061948.pth", "test20.csv")